In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.style as style
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
import statsmodels.api as sm

In [9]:
trim_df = pd.read_csv("outputs/trim_df.csv")
trim_df.drop(list(filter(lambda x: x.startswith('Unname'), trim_df)), axis = 1, inplace = True)

In [10]:
# Create correlation matrix
corr_matrix = trim_df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
trim_df.drop(to_drop, axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


In [11]:
trim_df=np.log(trim_df+1)
trim_df.isnull().values.any()

False

In [12]:
np.isinf(trim_df).values.any()

False

In [13]:
trim_df['NEUI']

0       2.564538e-06
1       1.870617e-05
2       5.102749e-06
3       3.656934e-05
4       2.315590e-05
            ...     
6496    2.565650e-05
6497    7.661046e-07
6498    6.864281e-05
6499    1.603222e-05
6500    1.383374e-05
Name: NEUI, Length: 6501, dtype: float64

In [14]:
y = trim_df['NEUI']
X = trim_df
X.drop('NEUI', axis=1, inplace=True)

In [15]:
## creating function to get model statistics
#import numpy as np
#import statsmodels.api as sm
#def get_stats():
#    x = X
#    results = sm.OLS(y, x).fit()
#    print(results.summary())
#get_stats()

In [16]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
def forward_selection(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<significance_level):
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

In [12]:
#forward_selection(X,y)